# Показатели каналов объединенных в группу
Пример расчета показателей по группе телеканалов.

## Описание задачи и условий расчета
- Период: 03.08.2022 - 09.08.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 14-44
- Место просмотра: Все места (дом+дача)
- Каналы: ТНТ, Пятница, ТВ-3, 2х2, ТНТ 4, Суббота, Матч ТВ (в группе и отдельно)									
- Статистики: AvReach000

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике телесетей получим id нужных нам каналов
cats.get_tv_net(name=["ТНТ", "ПЯТНИЦА", "ТВ-3", "2x2", "ТНТ 4", "СУББОТА", "МАТЧ ТВ"])
# Это id: 83, 204, 206, 286, 326, 329, 376

In [ ]:
# В справочнике демографических атрибутов найдем необходимую нам переменную
cats.get_tv_demo_attribute(names=['возраст'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: age

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-08-03', '2022-08-09')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА: Все 14-44
basedemo_filter = 'age >= 14 AND age <= 44'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = None

# Указываем список статистик для расчета
statistics = ['AvReach000']

# Указываем срезы
slices = []

# Задаем условия сортировки
sortings = None

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Сформируем словарь каналов

In [ ]:
nets = {
    "ТНТ, ПЯТНИЦА, ТВ-3, 2X2, МАТЧ ТВ, ТНТ 4, СУББОТА": '(83, 204, 206, 286, 326, 329, 376)',
    "ТНТ": '(83)',
    "ПЯТНИЦА": '(204)',
    "ТВ-3": '(206)',
    "2X2": '(286)',
    "МАТЧ ТВ": '(326)',
    "ТНТ 4": '(329)',
    "СУББОТА": '(376)'
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждого канала формируем задание и отправляем на расчет
for net_name, ids in nets.items():
    
    # Подставляем значения словаря в параметры
    project_name = net_name 
    company_filter = f"tvNetId IN {ids}"
              
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_timeband_task(task_name=project_name, date_filter=date_filter, 
                                          weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                          company_filter=company_filter, time_filter=time_filter, 
                                          basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                          location_filter=location_filter, slices=slices, 
                                          statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[['prj_name']+slices+statistics]
df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('03_joined_channels')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)